## Installing pre-reqs
Since this will be utilizing a PyTorch Paperspace Gradient, pytorch and its associated will be excluded from this.

In [ ]:
%pip install ultralytics wandb python-dotenv

# Importing Libraries
Ensure that you have installed the following libraries:
- Ultralytics YOLOv8
- PyTorch (Preferably CUDA version)

In [ ]:
from ultralytics import YOLO
from dotenv import load_dotenv
import torch
import os
import wandb

## Training the Model
This is training a new model from scratch. If you have a trained model or resuming training of a model, go to the section below.

In [ ]:
torch.multiprocessing.freeze_support()

# Loads .env config api.
load_dotenv()
WANDB_API_KEY = os.getenv('WANDB_API_KEY')
wandb.login(key=WANDB_API_KEY)
wandb.init(entity="Jonathan Turner", project=IcarusEye)

# Load a pretrained YOLOv5 model (recommended for transfer learning)
model = YOLO("yolov8s.pt")  # Use the YOLOv5 small version as the starting point

# Define hyperparameters to tweak
hyperparameters = {
    'lr0': 0.001,            # Initial learning rate
    'momentum': 0.95,       # Momentum for optimizer
    'weight_decay': 0.001, # Weight decay to avoid overfitting
    'batch_size': 24,       # Batch size (modify based on your GPU memory)
    'imgsz': 640,           # Image size for training
    'epochs': 200,          # Number of epochs
    'rect': True,           # Rectangular training for better performance on wide or tall images
    'hsv_h': 0.015,         # Color augmentation (hue)
    'hsv_s': 0.7,           # Color augmentation (saturation)
    'hsv_v': 0.4,           # Color augmentation (value)
    'flipud': 0.0,          # Vertical flip probability
    'fliplr': 0.5,          # Horizontal flip probability
    'perspective': 0.001,   # Perspective transformation (to simulate angle variation)
    'scale': 0.2,           # Scaling factor
    'mosaic': 0.3,          # Enable mosaic augmentation
    'mixup': 0.1            # Enable mixup augmentation
}

# Train the model
results = model.train(
    data="VisDrone.yaml",   # Path to your custom dataset YAML file
    epochs=hyperparameters['epochs'],
    imgsz=hyperparameters['imgsz'],
    batch=hyperparameters['batch_size'],
    lr0=hyperparameters['lr0'],
    momentum=hyperparameters['momentum'],
    weight_decay=hyperparameters['weight_decay'],
    augment=True,           # Enable augmentations
    perspective=hyperparameters['perspective'],
    scale=hyperparameters['scale'],
    mosaic=hyperparameters['mosaic'],
    mixup=hyperparameters['mixup'],
    hsv_h=hyperparameters['hsv_h'],
    hsv_s=hyperparameters['hsv_s'],
    hsv_v=hyperparameters['hsv_v'],
    fliplr=hyperparameters['fliplr'],
    rect=hyperparameters['rect'],
    optimizer='AdamW',
    resume=False           # Resume from last training session if True
)